In [1]:
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]
!git clone https://github.com/dsreload/sql_1

Cloning into 'sql_1'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 15 (delta 2), reused 5 (delta 2), pack-reused 3
Receiving objects: 100% (15/15), 4.01 MiB | 12.31 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [2]:
for table in table_names:
    data = pd.read_csv(f'sql_1/{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [3]:
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address FROM [Customers]
    WHERE Country in ('Germany', 'France') or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [4]:
pd.read_sql(
    '''
    SELECT Country, count(*) FROM [Customers]
    GROUP BY 1 ORDER BY 2 DESC LIMIT 3
    ''',
    con,
)

,Country,count(*)
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [5]:
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate FROM [Orders] o join [Shippers] s
    on o.ShipperID = s.ShipperID ORDER BY 2 LIMIT 1 OFFSET 9
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,02.10.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [6]:
pd.read_sql(
    '''
    SELECT OrderID, ProductName, (Quantity * Price) cost FROM [OrderDetails] o join [Products] p
    on p.ProductID = o.ProductID where OrderID = ( SELECT OrderID FROM [OrderDetails] o join [Products] p
                            on p.ProductID = o.ProductID GROUP BY 1 ORDER BY SUM(Quantity * Price) DESC LIMIT 1 )
    ''',
    con,
)

,OrderID,ProductName,cost
0,10372,Sir Rodney's Marmalade,972.0
1,10372,Côte de Blaye,10540.0
2,10372,Camembert Pierrot,2380.0
3,10372,Mozzarella di Giovanni,1461.6


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [7]:
pd.read_sql(
    '''
    SELECT OrderID, ProductName, Quantity FROM [OrderDetails] o join [Products] p
    on p.ProductID = o.ProductID where o.ProductID = ( SELECT ProductID FROM [OrderDetails] GROUP BY 1
                                                      ORDER BY SUM(Quantity) DESC LIMIT 1 )
    ''',
    con,
)

,OrderID,ProductName,Quantity
0,10253,Gorgonzola Telino,20
1,10272,Gorgonzola Telino,40
2,10273,Gorgonzola Telino,15
3,10325,Gorgonzola Telino,4
4,10335,Gorgonzola Telino,25
5,10342,Gorgonzola Telino,56
6,10356,Gorgonzola Telino,30
7,10359,Gorgonzola Telino,70
8,10363,Gorgonzola Telino,20
9,10374,Gorgonzola Telino,30


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [8]:
pd.read_sql(
    '''
    SELECT SupplierName, Country, Phone FROM OrderDetails o
    JOIN Products p ON o.ProductID = p.ProductID join Suppliers s on s.SupplierID = p.SupplierID
    GROUP BY 1 ORDER BY count(OrderID) DESC LIMIT 5
    ''',
    con,
)

,SupplierName,Country,Phone
0,Plutzer Lebensmittelgroßmärkte AG,Germany,(069) 992755
1,"Pavlova, Ltd.",Australia,(03) 444-2343
2,"Specialty Biscuits, Ltd.",UK,(161) 555-4448
3,Norske Meierier,Norway,(0)2-953010
4,Formaggi Fortini s.r.l.,Italy,(0544) 60323


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [9]:
pd.read_sql(
    '''
    SELECT CategoryName, Country, SUM(Price * Quantity) cost FROM OrderDetails od
    join Orders o ON od.OrderID = o.OrderID join Customers cu ON o.CustomerID = cu.CustomerID
    join Products p ON od.ProductID = p.ProductID join Categories c ON p.CategoryID = c.CategoryID
    where cu.Country = 'Brazil' GROUP BY 1 ORDER BY 3 DESC LIMIT 1
    ''',
    con,
)

,CategoryName,Country,cost
0,Beverages,Brazil,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [10]:
pd.read_sql(
    '''
    SELECT Country, max(Price * Quantity) - min(Price * Quantity) dif FROM OrderDetails od
    join Orders o on od.OrderID = o.OrderID join Customers c on o.CustomerID = c.CustomerID
    join Products p on od.ProductID = p.ProductID where Country = 'USA'
    ''',
    con,
)

,Country,dif
0,USA,5232.5


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [11]:
pd.read_sql(
    '''
    SELECT count(OrderID), FirstName || ' ' || LastName  FIO
    FROM Orders o join Employees e ON e.EmployeeID = o.EmployeeID
    GROUP BY o.EmployeeID ORDER BY BirthDate DESC LIMIT 3
    ''',
    con,
)

,count(OrderID),FIO
0,31,Janet Leverling
1,14,Robert King
2,40,Margaret Peacock


10. Сколько банок крабового мяса всего было заказано.

In [12]:
pd.read_sql(
    '''
    SELECT SUM(Quantity) * (SELECT SUBSTR(Unit, 6, 1) from Products where ProductName like '%Crab%') crabs
    FROM OrderDetails o join Products p ON o.ProductID = p.ProductID and ProductName like '%Crab%'
    ''',
    con,
)

,crabs
0,1024
